In [ ]:
from libs.Dataset import *

# TODO: tutto il progetto qui !!!


dm = TrashbinDataModule()
dm.setup()
dm.prepare_data()
# dm.setup(stage="fit")
# model = CustomModel()
# # model = Model(num_classes=dm.num_classes, width=dm.width, vocab=dm.vocab)
# trainer = pl.Trainer(max_epochs=1)
# trainer.fit(model, dm)